# 01 - Ingest PubMed Records

## Goal

Ingest PubMed records (dentistry focus): query by year using NCBI E-utilities, fetch MEDLINE XML into `data/raw/`. We'll be polite (rate limits), resumable, and explicit about scope.


## Why This Step Matters

**Reproducibility** is the foundation of scientific computing. By explicitly:

- Documenting our query terms
- Respecting NCBI rate limits (with `NCBI_EMAIL` and `NCBI_API_KEY`)
- Storing raw XML for provenance

...we ensure anyone can rebuild this dataset from scratch.

### How E-utilities Work

1. **ESearch** → returns list of PMIDs matching query
2. **EFetch** → retrieves full MEDLINE XML for those PMIDs

### Example XML Structure

```xml
<PubmedArticle>
  <MedlineCitation>
    <PMID>12345678</PMID>
    <Article>
      <ArticleTitle>Effect of dental implants...</ArticleTitle>
      <Abstract><AbstractText>This study...</AbstractText></Abstract>
    </Article>
    <MeshHeadingList>
      <MeshHeading><DescriptorName>Dental Implants</DescriptorName></MeshHeading>
    </MeshHeadingList>
  </MedlineCitation>
  <PubmedData>
    <PublicationTypeList>
      <PublicationType>Randomized Controlled Trial</PublicationType>
    </PublicationTypeList>
  </PubmedData>
</PubmedArticle>
```


In [36]:
# === TODO (you code this) ===
import os
import yaml
from pathlib import Path
from tqdm import tqdm
import sys
sys.path.append('..')
from src.utils import eutils_get
import os
from dotenv import load_dotenv


In [37]:
# === TODO (you code this) ===
# Goal: Read YAML config into a dict.
# Hints:
# 1) Use yaml.safe_load() on the file handle
# 2) Config should have: start_year, end_year, term_template, batch_ids, batch_fetch
# Acceptance:
# - config dict contains all 5 keys
# - start_year and end_year are integers

def load_config(path="../configs/query.yaml"):
    """Load YAML configuration."""
    # TODO
    with open(path, 'r') as f:
        config = yaml.safe_load(f)
        print(config["start_year"])
        print(config["end_year"])
        print(config["term_template"])
        print(config["batch_ids"])
        print(config["batch_fetch"])
    return config

config = load_config()


2018
2025
(dentistry[MeSH Terms] OR dental[Title/Abstract] OR "oral surgery"[Title/Abstract]
 OR periodont*[Title/Abstract] OR implant*[Title/Abstract]
 OR prosthodont*[Title/Abstract] OR endodont*[Title/Abstract]
 OR orthodont*[Title/Abstract] OR "maxillofacial"[Title/Abstract])
 AND ({{year}}[PDAT]:{{year}}[PDAT])

500
200


In [38]:
# === TODO (you code this) ===
# Goal: Set NCBI credentials from environment variables.
# Hints:
# 1) Use os.getenv() with sensible defaults
# 2) Without API key: 3 req/sec; with key: 10 req/sec
# Acceptance:
# - NCBI_EMAIL and NCBI_API_KEY variables defined
# - Defaults provided if env vars missing

# Set NCBI_EMAIL and NCBI_API_KEY with defaults
load_dotenv('../.env')
NCBI_EMAIL = os.getenv('NCBI_EMAIL', 'your_email@example.com')
NCBI_API_KEY = os.getenv('NCBI_API_KEY', None)

if not os.getenv('NCBI_EMAIL'):
    print("Warning: NCBI_EMAIL not found in environment variables. Using default.")
else:
    print("✅ NCBI_EMAIL found in environment variables.")

if not os.getenv('NCBI_API_KEY'):
    print("Warning: NCBI_API_KEY not found in environment variables. API rate limited to 3 req/sec.")
else:
    print("✅ NCBI_API_KEY found in environment variables.")


✅ NCBI_EMAIL found in environment variables.
✅ NCBI_API_KEY found in environment variables.


In [39]:
# === TODO (you code this) ===
# Goal: Create data/raw directory if it doesn't exist.
# Hints:
# 1) Use Path().mkdir() with appropriate flags
# Acceptance:
# - Directory exists after running
# - No error if directory already exists

# TODO: Create ../data/raw directory
if not os.path.exists('../data/raw'):
    os.makedirs('../data/raw')
    print("✅ Created ../data/raw directory.")
else:
    print("✅ ../data/raw directory already exists.")



✅ ../data/raw directory already exists.


## Build the Yearly Query

We'll loop through years and construct queries like:

```
(dentistry[MeSH Terms] OR dental[Title/Abstract] OR ...) AND (2018[PDAT]:2018[PDAT])
```


In [40]:
# === TODO (you code this) ===
# Goal: Build the PubMed query string for a given year.
# Hints:
# 1) Replace {{year}} placeholder in term_template
# 2) Return complete query string ready for ESearch
# Acceptance:
# - Function build_query(year:int, template:str) -> str
# - Query for 2022 contains "2022[PDAT]:2022[PDAT]"

def build_query(year, template):
    """Build PubMed query string for a given year."""
    # TODO
    query = template.replace('{{year}}', str(year))
    query = ' '.join(query.split())
    return query

# Test the function
test_query = build_query(2022, config['term_template'])
print(test_query)

# Verify it replaced both occurrences
assert '{{year}}' not in test_query  # Should pass - no more {{year}}
assert '2022' in test_query  # Should pass - year is in there
print("✅ build_query works correctly!")


(dentistry[MeSH Terms] OR dental[Title/Abstract] OR "oral surgery"[Title/Abstract] OR periodont*[Title/Abstract] OR implant*[Title/Abstract] OR prosthodont*[Title/Abstract] OR endodont*[Title/Abstract] OR orthodont*[Title/Abstract] OR "maxillofacial"[Title/Abstract]) AND (2022[PDAT]:2022[PDAT])
✅ build_query works correctly!


## ESearch: Get PMIDs

Use `esearch.fcgi` to get a list of PMIDs matching our query.


In [41]:
# === TODO (you code this) ===
# Goal: Call NCBI ESearch to get PMIDs for a query.
# Hints:
# 1) Use eutils_get() helper with 'esearch.fcgi' endpoint
# 2) Return JSON response (see NCBI E-utilities docs for params)
# 3) Include email and api_key in params for rate limit compliance
# Acceptance:
# - Function esearch(query, retmax, retstart) returns dict
# - Response contains 'esearchresult' with 'idlist'

def esearch(query, retmax=500, retstart=0):
    """Query NCBI ESearch and return JSON response."""
    params = {
        'db': 'pubmed',
        'term': query,
        'retmax': retmax,
        'retstart': retstart,
        'retmode': 'json',  # ⚠️ CRITICAL: Must specify JSON format
        'email': NCBI_EMAIL,
        'api_key': NCBI_API_KEY
    }
    
    # Debug: Print what we're sending
    print(f"🔍 Calling: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi")
    print(f"📝 Query preview: {query[:100]}...")
    
    response = eutils_get('esearch.fcgi', params=params)
    
    # Debug: Check what we got back
    print(f"✅ Status code: {response.status_code}")
    print(f"📦 Content type: {response.headers.get('content-type')}")
    print(f"📄 First 200 chars of response:\n{response.text[:200]}")
    
    return response.json()

# Test the function
# Test with a simple query first
test_query = build_query(2022, config['term_template'])
print(f"Query length: {len(test_query)} characters")

try:
    result = esearch(test_query, retmax=10, retstart=0)  # Start with just 10 results
    print(f"✅ Success! Found {result['esearchresult']['count']} results")
except Exception as e:
    print(f"❌ Error: {e}")




Query length: 295 characters
🔍 Calling: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi
📝 Query preview: (dentistry[MeSH Terms] OR dental[Title/Abstract] OR "oral surgery"[Title/Abstract] OR periodont*[Tit...
✅ Status code: 200
📦 Content type: application/json; charset=UTF-8
📄 First 200 chars of response:
{"header":{"type":"esearch","version":"0.3"},"esearchresult":{"count":"52879","retmax":"10","retstart":"0","idlist":["40654500","40654486","40453629","40224132","40224084","40182463","40182449","40110
✅ Success! Found 52879 results


In [43]:
# === TODO (you code this) ===
# Goal: Paginate through ESearch results to collect all PMIDs.
# Hints:
# 1) Loop with incrementing retstart until no more IDs returned
# 2) Use batch_size from config for retmax
# 3) Check total count to know when to stop
# Acceptance:
# - Function get_all_pmids(query, batch_size) -> list[str]
# - Returns all PMIDs, not just first batch
# - No duplicates

def get_all_pmids(query, batch_size=500):
    """Paginate ESearch to collect all PMIDs for a query."""
    pmids = []
    retstart = 0
    
    # First call to get total count
    first_result = esearch(query, retmax=batch_size, retstart=0)
    total_count = int(first_result['esearchresult']['count'])
    
    # NCBI hard limit: can only retrieve first 10,000 results
    MAX_RESULTS = 10000
    if total_count > MAX_RESULTS:
        print(f"⚠️  Found {total_count:,} results, but NCBI limits to {MAX_RESULTS:,}")
        print(f"    Will retrieve first {MAX_RESULTS:,} results")
        total_count = MAX_RESULTS
    else:
        print(f"📊 Found {total_count:,} total results")
    
    # Add first batch
    pmids.extend(first_result['esearchresult']['idlist'])
    retstart += batch_size
    
    # Progress bar
    from tqdm import tqdm
    pbar = tqdm(total=total_count, initial=batch_size, desc="Fetching PMIDs")
    
    # Continue pagination (with safety limit)
    while retstart < total_count and retstart < MAX_RESULTS:
        result = esearch(query, retmax=batch_size, retstart=retstart)
        id_list = result['esearchresult']['idlist']
        
        if not id_list:
            break
            
        pmids.extend(id_list)
        retstart += batch_size
        pbar.update(len(id_list))
    
    pbar.close()
    
    print(f"✅ Collected {len(pmids):,} unique PMIDs")
    return pmids

# Test with 2024 (smaller dataset)
test_query = build_query(2024, config['term_template'])
pmids_2024 = get_all_pmids(test_query, batch_size=500)

# Verify
print(f"\nFirst 5 PMIDs: {pmids_2024[:5]}")
print(f"Last 5 PMIDs: {pmids_2024[-5:]}")
print(f"Any duplicates? {len(pmids_2024) != len(set(pmids_2024))}")


🔍 Calling: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi
📝 Query preview: (dentistry[MeSH Terms] OR dental[Title/Abstract] OR "oral surgery"[Title/Abstract] OR periodont*[Tit...
✅ Status code: 200
📦 Content type: application/json; charset=UTF-8
📄 First 200 chars of response:
{"header":{"type":"esearch","version":"0.3"},"esearchresult":{"count":"55143","retmax":"500","retstart":"0","idlist":["41194911","41134706","41113858","41101911","41069390","41064704","41059869","4105
⚠️  Found 55,143 results, but NCBI limits to 10,000
    Will retrieve first 10,000 results


🔍 Calling: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi
📝 Query preview: (dentistry[MeSH Terms] OR dental[Title/Abstract] OR "oral surgery"[Title/Abstract] OR periodont*[Tit...


✅ Status code: 200
📦 Content type: application/json; charset=UTF-8
📄 First 200 chars of response:
{"header":{"type":"esearch","version":"0.3"},"esearchresult":{"count":"55143","retmax":"500","retstart":"500","idlist":["40134401","40134400","40134399","40134398","40134397","40134395","40134393","40
🔍 Calling: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi
📝 Query preview: (dentistry[MeSH Terms] OR dental[Title/Abstract] OR "oral surgery"[Title/Abstract] OR periodont*[Tit...


✅ Status code: 200
📦 Content type: application/json; charset=UTF-8
📄 First 200 chars of response:
{"header":{"type":"esearch","version":"0.3"},"esearchresult":{"count":"55143","retmax":"500","retstart":"1000","idlist":["39947775","39947774","39947773","39947772","39947770","39947112","39945008","3
🔍 Calling: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi
📝 Query preview: (dentistry[MeSH Terms] OR dental[Title/Abstract] OR "oral surgery"[Title/Abstract] OR periodont*[Tit...


✅ Status code: 200
📦 Content type: application/json; charset=UTF-8
📄 First 200 chars of response:
{"header":{"type":"esearch","version":"0.3"},"esearchresult":{"count":"55143","retmax":"500","retstart":"1500","idlist":["39874511","39874334","39874323","39873262","39873261","39873260","39873257","3
🔍 Calling: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi
📝 Query preview: (dentistry[MeSH Terms] OR dental[Title/Abstract] OR "oral surgery"[Title/Abstract] OR periodont*[Tit...


...

✅ Status code: 200
📦 Content type: application/json; charset=UTF-8
📄 First 200 chars of response:
{"header":{"type":"esearch","version":"0.3"},"esearchresult":{"count":"55143","retmax":"500","retstart":"8000","idlist":["39556665","39556628","39556561","39556553","39556459","39556351","39556329","3
🔍 Calling: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi
📝 Query preview: (dentistry[MeSH Terms] OR dental[Title/Abstract] OR "oral surgery"[Title/Abstract] OR periodont*[Tit...


✅ Status code: 200
📦 Content type: application/json; charset=UTF-8
📄 First 200 chars of response:
{"header":{"type":"esearch","version":"0.3"},"esearchresult":{"count":"55143","retmax":"500","retstart":"8500","idlist":["39539922","39539873","39539870","39539869","39539854","39539852","39539850","3
🔍 Calling: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi
📝 Query preview: (dentistry[MeSH Terms] OR dental[Title/Abstract] OR "oral surgery"[Title/Abstract] OR periodont*[Tit...


✅ Status code: 200
📦 Content type: application/json; charset=UTF-8
📄 First 200 chars of response:
{"header":{"type":"esearch","version":"0.3"},"esearchresult":{"count":"55143","retmax":"500","retstart":"9000","idlist":["39523191","39523189","39523105","39523089","39523037","39523009","39522987","3
🔍 Calling: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi
📝 Query preview: (dentistry[MeSH Terms] OR dental[Title/Abstract] OR "oral surgery"[Title/Abstract] OR periodont*[Tit...




Fetching PMIDs: 100%|█████████▉| 9999/10000 [00:17<00:00, 544.29it/s]

✅ Status code: 200
📦 Content type: application/json; charset=UTF-8
📄 First 200 chars of response:
{"header":{"type":"esearch","version":"0.3"},"esearchresult":{"count":"55143","retmax":"499","retstart":"9500","idlist":["39506764","39506746","39506742","39506741","39506722","39506543","39506497","3
✅ Collected 9,999 unique PMIDs

First 5 PMIDs: ['41194911', '41134706', '41113858', '41101911', '41069390']
Last 5 PMIDs: ['39489114', '39489075', '39489029', '39489006', '39488890']
Any duplicates? False


## EFetch: Download XML

Use `efetch.fcgi` to retrieve MEDLINE XML for batches of PMIDs.


In [ ]:
# === TODO (you code this) ===
# Goal: Fetch MEDLINE XML for a list of PMIDs.
# Hints:
# 1) Join PMIDs with commas for 'id' parameter
# 2) Request 'xml' retmode (not json)
# 3) Return raw XML text, not JSON
# Acceptance:
# - Function efetch(pmids:list) -> str
# - Returns XML string starting with <?xml

def efetch(pmids):
    """Fetch MEDLINE XML for a batch of PMIDs."""
    # TODO: build params and call eutils_get, return .text
    raise NotImplementedError


## Main Ingestion Loop

For each year:
1. Build query
2. Get PMIDs
3. Fetch XML in batches
4. Save to `data/raw/pubmed_{year}_{batch}.xml`


In [ ]:
# === TODO (you code this) ===
# Goal: Orchestrate full ingestion for all years.
# Hints:
# 1) Loop through year range from config
# 2) For each year: build query, get all PMIDs, chunk and fetch XML
# 3) Save files as pubmed_{year}_{chunk:04d}.xml
# 4) Skip existing files for resumability
# Acceptance:
# - Function ingest_years(config) processes all years
# - Files written to ../data/raw/
# - Re-running skips existing files
# - Progress printed for each year

def ingest_years(config):
    """Ingest all configured years into data/raw/."""
    # TODO: implement year loop with chunked fetching
    raise NotImplementedError

# Run ingestion (uncomment when ready)
# ingest_years(config)


## QA Checklist

Before moving to the next notebook, verify:

- [ ] Counts per year are non-zero
- [ ] Files written to `data/raw/` directory
- [ ] XML files are valid (spot-check by opening one)
- [ ] Re-running the cell skips existing files (resumability)
- [ ] No rate-limit errors from NCBI

### Sanity Check: Count Files


In [ ]:
# === TODO (you code this) ===
# Goal: Count and verify downloaded XML files.
# Hints:
# 1) Use Path.glob() to find all .xml files
# 2) Parse filenames to extract year
# 3) Show total count and breakdown by year
# Acceptance:
# - Prints total file count
# - Shows count per year (use Counter)

# TODO: count files in ../data/raw/*.xml


## 🧘 Reflection Log

**What did you learn in this session?**
- 

**What challenges did you encounter?**
- 

**How will this improve Periospot AI?**
- 
